In [ ]:
import numpy as np
import pandas as pd

from Utils import extract_symbol_timeframe


# === EMA и SMA ===
def ema(arr, window):
    alpha = 2 / (window + 1)
    result = np.empty_like(arr)
    result[:] = np.nan
    result[0] = arr[0]
    for i in range(1, len(arr)):
        result[i] = alpha * arr[i] + (1 - alpha) * result[i - 1]
    return result

def sma(arr, window):
    return pd.Series(arr).rolling(window=window).mean().to_numpy()

# === Zero Lag MACD Enhanced ===
def compute_zero_lag_macd(close, fast_len, slow_len, signal_len, use_ema=True, use_old_algo=False):
    # Fast line
    ma1 = ema(close, fast_len) if use_ema else sma(close, fast_len)
    ma2 = ema(ma1, fast_len) if use_ema else sma(ma1, fast_len)
    zero_fast = 2 * ma1 - ma2

    # Slow line
    mas1 = ema(close, slow_len) if use_ema else sma(close, slow_len)
    mas2 = ema(mas1, slow_len) if use_ema else sma(mas1, slow_len)
    zero_slow = 2 * mas1 - mas2

    # MACD line
    zero_lag_macd = zero_fast - zero_slow

    # Signal line
    if use_old_algo:
        signal = sma(zero_lag_macd, signal_len)
    else:
        emasig1 = ema(zero_lag_macd, signal_len)
        emasig2 = ema(emasig1, signal_len)
        signal = 2 * emasig1 - emasig2

    return zero_lag_macd, signal

# === Сделки по правилам ===
def extract_trades(df, macd, signal, tp=9.0, stop_loss_pct=99):
    macd_shift = np.roll(macd, 1)
    signal_shift = np.roll(signal, 1)

    # === Пересечение снизу вверх
    cross_up = (macd_shift < signal_shift) & (macd > signal)
    cross_up[0] = False

    # === Фиксированный 75-й перцентиль
    perc_99 = 0 - np.percentile(macd[~np.isnan(macd)], 99)

    # Условие входа: пересечение снизу + MACD ниже 75 перц.
    valid_entry = cross_up & (macd < perc_99)

    close = df["close"].to_numpy()
    high = df["high"].to_numpy()
    low = df["low"].to_numpy()
    open_time = df["open_time"].to_numpy()

    trades = []
    last_exit = 0

    for i in np.where(valid_entry)[0]:
        if i <= last_exit:
            continue

        entry_price = close[i]
        take_profit = entry_price * (1 + tp / 100)
        stop_loss = entry_price * (1 - stop_loss_pct / 100)
        entry_time = open_time[i]

        stop_triggered = False
        stop_time = pd.NaT

        for j in range(i + 1, len(close)):
            if high[j] >= take_profit:
                exit_price = take_profit
                exit_time = open_time[j]
                break
            elif low[j] <= stop_loss:
                exit_price = stop_loss
                exit_time = open_time[j]
                stop_triggered = True
                stop_time = open_time[j]
                break
        else:
            break

        duration = int((exit_time - entry_time).total_seconds() // 60)

        trades.append({
            "entry_time": entry_time,
            "entry_price": entry_price,
            "macd": macd[i],
            "macd_signal": signal[i],
            "exit_time": exit_time,
            "exit_price": exit_price,
            "duration_min": duration,
            "stop_triggered": stop_triggered,
            "stop_time": stop_time
        })

        last_exit = j

    return pd.DataFrame(trades)

# === Загрузка и расчёт ===

path = " " # Format filename = {symbol}USDT_timeframe_data.csv, example: "BTCUSDT_1h_from_1_Jan_2020.csv"
df = pd.read_csv(path)

symbol, timeframe = extract_symbol_timeframe(path)
indicator_name = "ZeroLag_MACD_with_logic_99prct"

df["open_time"] = pd.to_datetime(df["open_time"], utc=True)
df.dropna(inplace=True)

macd, signal = compute_zero_lag_macd(
    close=df["close"].to_numpy(dtype=np.float64),
    fast_len=6,
    slow_len=22,
    signal_len=5,
    use_ema=True,
    use_old_algo=False
)

# Обрезаем фрейм под размер массива
df = df.iloc[len(df) - len(macd):].copy()
df["macd"] = macd
df["macd_signal"] = signal

tp = 9.0

deals_df = extract_trades(df, macd, signal, tp=tp, stop_loss_pct=99)

# === Вывод ===
print("✅ Сделок найдено:", len(deals_df))
print(f"Криптовалюта: {symbol}\nТаймфрейм: {timeframe}\nСтратегия: {indicator_name}\nПрофит: {tp}%")
print("📊 99-й перцентиль MACD:", round(np.percentile(macd[~np.isnan(macd)], 99), 5))
deals_df.tail(10)
